# Visualizing prob-maps

In [ ]:
import pickle
import itertools
import glob
import gc
import random

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from tqdm.auto import tqdm, trange
from statsmodels.stats.proportion import proportion_confint

from pyecsca.ec.mult import *
from pyecsca.misc.utils import TaskExecutor

from common import *

%matplotlib ipympl

## Setup
Setup some plotting and the computations of prob-maps out of the small scalar data and divisors.

In [ ]:
# Setup the ticks and colors deterministically.
mult_klasses = sorted(list(set(map(lambda mult: mult.klass, all_mults))), key=lambda klass: klass.__name__)
mult_kwarg_map = {klass: 0 for klass in mult_klasses}
mult_cm_map = {mult: 0 for mult in all_mults}
mult_colors = matplotlib.cm.tab20(range(len(mult_klasses)))
mult_styles = ['-', '--', '-.', ':', (5, (10, 3)), (0, (5, 1)), (0, (3, 1, 1, 1, 1, 1)), (0, (3, 1, 1, 1)), (0, (1, 1)), (0, (3, 10, 1, 10))]
mult_markers = [None, "o", "+", "*", "^", "s"]
colors = {}
styles = {}
markers = {}
for mult in all_mults:
    color = mult_colors[mult_klasses.index(mult.klass) % len(mult_colors)]
    style = mult_styles[mult_kwarg_map[mult.klass] % len(mult_styles)]
    mult_kwarg_map[mult.klass] += 1
    for cm in (None, "gsr", "additive", "multiplicative", "euclidean", "bt"):
        mwc = mult.with_countermeasure(cm)
        colors[mwc] = color
        styles[mwc] = style
        markers[mwc] = mult_markers[mult_cm_map[mult] % len(mult_markers)]
        mult_cm_map[mult] += 1

majticks = np.arange(0, 1, 0.1)
minticks = np.arange(0, 1, 0.05)

## Divisors
The cell below contains some interesting divisors for distinguishing scalarmults.

In [ ]:
from common import divisor_map
for d, ds in divisor_map.items():
    print(f"{d:<27}", ds[:3], "...", ds[-1:])

In [ ]:
bits = 256
num_workers = 28

## Configuration
Select the mults you want to compute the prob-maps for here as well as a set of divisors. It is good to set `all` here, compute the prob-maps for all the divisors, save them and they continue with visualizing them on subsets of divisors.

In [ ]:
selected_mults = all_mults
divisor_name = "all"
showci = False
selected_divisors = divisor_map[divisor_name]
show_error_model = ErrorModel({}, "all", True)

In [ ]:
with open(f"merged.pickle", "rb") as f:
    distributions_mults = pickle.load(f)

Now, go over all the divisor sets and visualize them (without the combs) into PNGs in the graphs/ directory.

## Plots (nocomb)
Let's visualize all the divisor groups while looking at the multipliers and countermeasures except the comb-like ones.

In [ ]:
for divisor_name in divisor_map:
    plot_mults = list(filter(lambda mult: mult in distributions_mults and mult.klass not in (CombMultiplier, BGMWMultiplier), distributions_mults))
    print(divisor_name, "nocomb")
    plot_divisors = sorted(divisor_map[divisor_name])
    L = len(plot_divisors)
    N = len(plot_mults)
    x = list(range(L))
    
    fig = plt.figure(figsize=(L/4+10, 24))
    ax = plt.subplot(111)
    
    vals = np.zeros((N, L))
    n_samples = 0
    for i, mult in enumerate(plot_mults):
        clear_mult = mult.with_error_model(None)
        probmap = distributions_mults[mult]
        y_values = [probmap[l] for l in plot_divisors]
        vals[i,] = y_values
        ax.plot(x, y_values,
                color=colors[clear_mult],
                linestyle=styles[clear_mult],
                marker=markers[clear_mult],
                label=str(mult) if mult.countermeasure is None and mult.error_model == show_error_model else "_nolegend_")
        if showci:
            cis = [conf_interval(p, probmap.samples) for p in y_values]
            ci_low = [ci[0] for ci in cis]
            ci_high = [ci[1] for ci in cis]
            ax.fill_between(x, ci_low, ci_high, color="black", alpha=0.1)
        n_samples += probmap.samples
    
    ax.set_title(f"{divisor_name}\nSamples: " + str(n_samples//N))
    
    #var = np.var(vals, axis=0)
    #ax.plot(x, var / np.max(var), label="cross-mult variance (normalized)", ls="--", lw=2, color="black")
    
    ax.set_xlabel("divisors")
    ax.set_ylabel("error probability")
    ax.set_yticks(majticks)
    ax.set_yticks(minticks, minor=True)
    ax.set_xticks(x, plot_divisors, rotation=90)
    
    ax.grid(axis="y", which="major", alpha=0.7)
    ax.grid(axis="y", which="minor", alpha=0.3)
    ax.grid(axis="x", alpha=0.7)
    plt.tight_layout()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
    
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    fig.savefig(f"graphs/{divisor_name}-nocomb{'+ci' if showci else ''}.pdf");
    plt.close()

## Plots (allmults)
Now, lets also do plots with allmults for all divisor groups.

In [ ]:
for divisor_name in divisor_map:
    plot_mults = list(filter(lambda mult: mult in distributions_mults, all_mults_with_ctr))
    print(divisor_name, "allmults")
    plot_divisors = sorted(divisor_map[divisor_name])
    L = len(plot_divisors)
    N = len(plot_mults)
    x = list(range(L))
    
    fig = plt.figure(figsize=(L/4+10, 26))
    ax = plt.subplot(111)
    
    vals = np.zeros((N, L))
    n_samples = 0
    for i, mult in enumerate(plot_mults):
        probmap = distributions_mults[mult]
        y_values = [probmap[l] for l in plot_divisors]
        vals[i,] = y_values
        ax.plot(x, y_values,
                color=colors[mult],
                linestyle=styles[mult],
                marker=markers[mult],
                label=str(mult) if mult.countermeasure is None else "_nolegend_")
        if showci:
            cis = [conf_interval(p, probmap.samples) for p in y_values]
            ci_low = [ci[0] for ci in cis]
            ci_high = [ci[1] for ci in cis]
            ax.fill_between(x, ci_low, ci_high, color="black", alpha=0.1)
        n_samples += probmap.samples
    
    ax.set_title(f"{divisor_name} ({kind})\nSamples(avg): " + str(n_samples//N))
    
    #var = np.var(vals, axis=0)
    #ax.plot(x, var / np.max(var), label="cross-mult variance (normalized)", ls="--", lw=2, color="black")
    
    ax.set_xlabel("divisors")
    ax.set_ylabel("error probability")
    ax.set_yticks(majticks)
    ax.set_yticks(minticks, minor=True)
    ax.set_xticks(x, plot_divisors, rotation=90)
    
    ax.grid(axis="y", which="major", alpha=0.7)
    ax.grid(axis="y", which="minor", alpha=0.3)
    ax.grid(axis="x", alpha=0.7)
    plt.tight_layout()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
    
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    fig.savefig(f"graphs/{kind}-kind/{divisor_name}-allmults{'+ci' if showci else ''}.pdf")
    plt.close()

## Interactive plot
Below you can choose a concrete divisor set and visualize it with all the mults, or just some to your liking.

In [ ]:
#divisor_name = "powers_of_2_large"
single_mult = random.choice(list(distributions_mults.keys())).with_error_model(None)
divisor_name = "all_integers"
plot_mults = list(filter(lambda mult: mult.with_error_model(None) == single_mult, distributions_mults))
plot_divisors = divisor_map[divisor_name]
#plot_divisors = (61, 65, 111, 165, 1536, 12288) 
#plot_divisors = (55, 65, 165, 248, 3072)
L = len(plot_divisors)
N = len(plot_mults)
x = list(range(L))

colors = plt.get_cmap('tab20').colors +plt.get_cmap('tab20b').colors[:12]

fig = plt.figure(figsize=(L/4+15, 24))
ax = plt.subplot(111)

vals = np.zeros((N, L))
n_samples = 0
groups = {}
for i, mult in enumerate(plot_mults):

    clear_mult = mult.with_error_model(None)
    probmap = distributions_mults[mult]
    y_values = [probmap[l] for l in plot_divisors]
    y_tup = tuple(y_values)
    group = groups.setdefault(y_tup, set())
    group.add(mult)
    vals[i,] = y_values
    offset = (i - N/2) * 0.0001
    ax.plot(x,[v + offset for v in y_values],
            color=colors[i],
            linestyle=styles[clear_mult],
            marker=markers[clear_mult],
            label=str(mult))
    if showci:
        cis = [conf_interval(p, probmap.samples) for p in y_values]
        ci_low = [ci[0] for ci in cis]
        ci_high = [ci[1] for ci in cis]
        ax.fill_between(x, ci_low, ci_high, color="black", alpha=0.1)
    n_samples += probmap.samples

ax.set_title(f"{divisor_name}\nSamples(avg): " + str(n_samples//N))

#var = np.var(vals, axis=0)
#ax.plot(x, var / np.max(var), label="cross-mult variance (normalized)", ls="--", lw=2, color="black")

ax.set_xlabel("divisors")
ax.set_ylabel("error probability")
ax.set_yticks(majticks)
ax.set_yticks(minticks, minor=True)
ax.set_xticks(x, plot_divisors, rotation=90)

ax.grid(axis="y", which="major", alpha=0.7)
ax.grid(axis="y", which="minor", alpha=0.3)
ax.grid(axis="x", alpha=0.7)
plt.tight_layout()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5));
for _, group in groups.items():
    print(group)
    print()

In [ ]:
divisor_name = "all"
plot_divisors = divisor_map[divisor_name]
ngroups = []
for mult in all_mults_with_ctr:
    groups = {}
    for error_model in all_error_models:
        full = mult.with_error_model(error_model)
        probmap = distributions_mults[full]
        y_values = [probmap[l] for l in plot_divisors]
        y_tup = tuple(y_values)
        group = groups.setdefault(y_tup, set())
        group.add(mult)
    ngroups.append(len(groups))
print(np.min(ngroups))
print(np.mean(ngroups))
print(np.median(ngroups))
print(np.max(ngroups))

In [ ]:
plt.close("all")